# RUN: CoV Postive Controls II: errors

```
Lead     : ababaian
Issue    : na
Version  : 5f03d2fe21ae31e285e7c88854514af1d9bf5d71
start    : 2020 05 02
complete : 2020 -- --
files    : ~/serratus/notebook/200502_ab/
s3_files : s3://serratus-public/notebook/200502_ab/
output   : s3://serratus-public/out/200427_cov_positive/
```

### Objectives
- The previous run [200427_Run_CoV_Positive_Control](200427_Run_CoV_Positive_Control.ipynb) had some errors.
- This run will re-try the previous runs with errors


Note: I also realized a commit error meant that most of the previous runs were aligned against `cov1r` not `cov2r` pan-genomes. So output in the end is a mixture of these two pan-genomes.


## Serratus Initialization

Local system initialization procedures for `serratus`

In [1]:
date

Sun May  3 18:43:34 PDT 2020


### Initialize local workspace

In [2]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS
git rev-parse HEAD # commit version

3f5911215db222f02e2130ce6bfb56b51ae2ccfe


In [57]:
# Create local run directory
SERRATUS="/home/artem/serratus"
WORK="$SERRATUS/notebook/200502_ab"
mkdir -p $WORK; cd $WORK

### SRA Run Info for error files

Check which files had errors by which files did not have correct `bai` files generated for them.


In [4]:
# total input sra
cp ../200427_ab/CoV_positive_SraRunInfo.csv ./
wc -l CoV_positive_SraRunInfo.csv

872 CoV_positive_SraRunInfo.csv


In [5]:
# completed files
aws s3 ls s3://serratus-public/out/200427_CoV_positve_ctrl/bai/ | wc -l

740


In [6]:
# Generate SraRunInfo for errors
aws s3 ls \
  s3://serratus-public/out/200427_CoV_positve_ctrl/bai/ \
  | cut -f 8 -d' ' - \
  | cut -f 1 -d'.' - > sra.tmp

cp CoV_positive_SraRunInfo.csv CoV_positive_run_ii.SraRunInfo.csv

for LINE in $(cat sra.tmp)
do
  sed -i "/$LINE/d" CoV_positive_run_ii.SraRunInfo.csv
done

wc -l CoV_positive_run_ii.SraRunInfo.csv

rm CoV_positive_SraRunInfo.csv

132 CoV_positive_run_ii.SraRunInfo.csv


In [7]:
# SRA RunInfo Table for run
RUNINFO="$WORK/CoV_positive_run_ii.SraRunInfo.csv"
cat $RUNINFO

Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,Experiment,LibraryName,LibraryStrategy,LibrarySelection,LibrarySource,LibraryLayout,InsertSize,InsertDev,Platform,Model,SRAStudy,BioProject,Study_Pubmed_id,ProjectID,Sample,BioSample,SampleType,TaxID,ScientificName,SampleName,g1k_pop_code,source,g1k_analysis_group,Subject_ID,Sex,Disease,Tumor,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
SRR11491770,2020-04-07 11:39:15,2020-04-13 15:12:16,1438700,375053251,1438700,260,222,,https://sra-download.ncbi.nlm.nih.gov/traces/sra80/SRR/011222/SRR11491770,SRX8067711,UT-00291,AMPLICON,PCR,VIRAL RNA,PAIRED,0,0,ILLUMINA,Illumina MiSeq,SRP253926,PRJNA614995,,614995,SRS6436363,SAMN14546137,simple,2697049,Severe acute respiratory syndrome coronavirus 2,UT-00291,,,,,,,no,,,,,UTAH PUBLIC HEALTH LABORATORY,SRA1063178,,public,3D40CD885CAE4125861C8F89D9A72E90,3B5B4DD41E6E177992D0

### Packer / AMI Initialization (optional)
Does not need to be ran each time if you have access to the AMI already.

Current Build: `us-east-1: ami-046baafb2ee438b69`

In [ ]:
cd $SERRATUS/packer
packer build docker-ami.json

### Build Serratus containers (optional)
Serratus containers are available on the `serratusbio` dockerhub. If you wish to deploy your own containers, you will have to build them from the `serratus` repository and upload them to your own dockerhub account.

This can be done with the `build.sh` script

In [ ]:
cd $SERRATUS

# If you want to upload containers to your repository
# include this.
export DOCKERHUB_USER='serratusbio' # optional
sudo docker login # optional

# Build all containers and upload them docker hub repo
# (if available)
./container_build.sh

### Terraform Initialization
The Global Variables for Terraform file must be modified to initialize for your system.

File: `$SERRATUS/terarform/main/terraform.tfvars`

This step must be done manually in a text editor currently.

In [ ]:
## Change these parameters in
## $SERRATUS/terarform/main/terraform.tfvars

# Your public IP followed by "/32"
# use: `curl ipecho.net/plain; echo`
LOCALIP="75.155.242.67/32" #dev_cidrs
# Your AWS key name
KEYNAME="serratus"         #key_name
# Dockerhub account containing serratus containers
DOCKERHUB_USER='serratusbio'    #dockerhub_account (optional)

In [ ]:
## CUSTOM CHANGES TO TERRAFORM
## $SERRATUS/terarform/main/main.tf
##
## main.tf
## Line 171:   options            = "-k ${module.work_bucket.name} -b s3://serratus-public/out/200427_CoV_positve_ctrl"
##


In [8]:
# Initialize terraform
cd $SERRATUS/terraform/main
terraform init

Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corresponding provider blocks in configuration, with the constraint strings
suggested below.

* provider.local: version = "~> 1.4"

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [9]:
# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

module.download.data.aws_availability_zones.all: Refreshing state...
module.merge.data.aws_ami.amazon_linux_2: Refreshing state...
module.align.data.aws_ami.amazon_linux_2: Refreshing state...
module.align.data.aws_region.current: Refreshing state...
module.scheduler.data.aws_region.current: Refreshing state...
module.scheduler.data.aws_ami.amazon_linux_2: Refreshing state...
module.align.data.aws_availability_zones.all: Refreshing state...
module.download.data.aws_ami.amazon_linux_2: Refreshing state...
module.merge.data.aws_region.current: Refreshing state...
module.merge.data.aws_availability_zones.all: Refreshing state...
module.download.data.aws_region.current: Refreshing state...
module.monitoring.data.aws_ami.ecs: Refreshing state...
module.monitoring.aws_ecs_cluster.monitor: Creating...
module.download.aws_cloudwatch_log_group.g: Creating...
module.scheduler.aws_cloudwatch_log_group.scheduler: Creating...
module.merge.aws_cloudwatch_log_group.g: Creating...
module.align.aws_clo

## Running Serratus 
Upload the run data, scale-out the cluster, monitor performance.


### Run Monitors & Upload table

Open SSH tunnels to monitor node then open monitors in browser


In [10]:
cd $SERRATUS/terraform/main
sleep 30s

# Open SSH tunnels to the monitor
./create_tunnels.sh

Tunnels created:

    localhost:3000 -- grafana

    localhost:9090 -- prometheus

    localhost:8000 -- scheduler


In [11]:
# Load SRA Run Info into scheduler (READY)
curl -s -X POST -T $RUNINFO localhost:8000/jobs/add_sra_run_info/

{
  "inserted_rows": 130, 
  "total_rows": 130
}


### Scale up the cluster
This will set-up 5 download, 5 align and 2 merge nodes to process data


In [42]:
./dl_set_capacity.sh 1

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-dl-20200504014510569900000009 --desired-capacity 1


In [55]:
./align_set_capacity.sh 1

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-align-2020050401451057050000000a --desired-capacity 1


In [53]:
# For scaling up larger use step-wise ASG increases
for ASG_CAP in $(seq 12 2 40)
do
  #echo Set DesiredSize: $ASG_CAP
  ./align_set_capacity.sh $ASG_CAP
  sleep 60s
done

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-align-2020050401451057050000000a --desired-capacity 12
+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-align-2020050401451057050000000a --desired-capacity 14
+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-align-2020050401451057050000000a --desired-capacity 16
+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-align-2020050401451057050000000a --desired-capacity 18
+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-align-2020050401451057050000000a --desired-capacity 20
+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacit

In [38]:
./merge_set_capacity.sh 1

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-merge-20200504014510541400000008 --desired-capacity 1


In [ ]:
# Error fixes (manually help along)
#curl -X POST "localhost:8000/jobs/split/601?state=split_done&N_paired=73&N_unpaired=0"

# Error fixes (stalled run) SRR11059947
#curl -X POST "localhost:8000/jobs/split/54?state=new&N_paired=0&N_unpaired=0"

# Error fixes (stalled run) SRR11059947
#BLOCK_ID=1

for BLOCK_ID in $(seq 492 493)
do
  curl -X POST -s "localhost:8000/jobs/align/$BLOCK_ID?state=new"
done

for ACC in $(seq 24 33)
do
  curl -X POST "localhost:8000/jobs/split/$ACC?state=new&N_paired=0&N_unpaired=0"
done

# One Liner to reset range of Alignment Blocks
X=650; Y=650;STATE='new'; for BLOCK_ID in $(seq $X $Y); do  curl -X POST -s "localhost:8000/jobs/align/$BLOCK_ID?state=$STATE"; done


## Shutting down procedures

Closing up shop.

In [62]:
# When all downloading/splitting is done,
# scale-in the downloaders
cd $SERRATUS/terraform/main
./dl_set_capacity.sh 0

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-dl-20200504014510569900000009 --desired-capacity 0


In [63]:
# When all alignment is done,
# scale-in the aligners
./align_set_capacity.sh 0

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-align-2020050401451057050000000a --desired-capacity 0


In [64]:
# When all merging is done,
# scale in the mergers
./merge_set_capacity.sh 0

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name serratus-merge-20200504014510541400000008 --desired-capacity 0


In [59]:
# Dump the Scheduler SQLITE table to a local file
curl localhost:8000/db > \
  $WORK/schedDump.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  704k  100  704k    0     0   816k      0 --:--:-- --:--:-- --:--:--  815k


### Save output of runs


## Destroy Cluster

Close out all resources with terraform (will take a few minutes).


In [65]:
terraform destroy -auto-approve
# WARNING this will also delete the standard output bucket/data
# Save data prior to destroy

module.merge.aws_cloudwatch_log_group.g: Refreshing state... [id=serratus-merge]
module.merge.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-serratus-merge]
module.scheduler.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-scheduler]
module.align.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-serratus-align]
module.monitoring.aws_iam_role.task_role: Refreshing state... [id=SerratusIamRole-monitor]
module.merge.data.aws_region.current: Refreshing state...
module.download.data.aws_availability_zones.all: Refreshing state...
module.download.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-serratus-dl]
module.work_bucket.aws_s3_bucket.work: Refreshing state... [id=tf-serratus-work-20200504014443329100000001]
module.monitoring.aws_ecs_cluster.monitor: Refreshing state... [id=arn:aws:ecs:us-east-1:797308887321:cluster/serratus-monitor]
module.align.aws_cloudwatch_log_group.g: Refreshin

# Run Notes
Notes on this run for future reference.

## Errors

### OC43 Amplicon Sequencing - Repeated Align Crashes
The runs from the experiment `PRJNA507154` are stalling out the aligners which are getting termination signals and/or some unknown status check is killing them. Only the last block (smaller files) are completing, these can therefore be cleaned up by making fq-block smaller and runtime of the alignment will be smaller. Consider a special run with 50,000 fq-blocks.

Accessions: SRR8265747, SRR8265748, SRR8265749, SRR8265750, SRR8265751, SRR8265752, SRR8265753, SRR8265755

### SRA unavailable
SRA accessions that could not be downloaded (retried twice). SRA is unavailable, note the rapid failure time in each of these cases.

```
6	ERR3486809	split_err	2020-05-04 01:48:43.844369	2020-05-04 01:48:44.297944	i-077da150eefd54064-1	None	None	None
7	ERR3489114	split_err	2020-05-04 01:48:44.345634	2020-05-04 01:48:44.775971	i-077da150eefd54064-1	None	None	None
8	ERR3489845	split_err	2020-05-04 01:48:44.813044	2020-05-04 01:48:45.047149	i-077da150eefd54064-1	None	None	None
9	ERR3489846	split_err	2020-05-04 01:48:45.078254	2020-05-04 01:48:45.375405	i-077da150eefd54064-1	None	None	None
10	ERR3489847	split_err	2020-05-04 01:48:45.404139	2020-05-04 01:48:45.691091	i-077da150eefd54064-1	None	None	None
11	ERR3489104	split_err	2020-05-04 01:48:45.727474	2020-05-04 01:48:45.975516	i-077da150eefd54064-1	None	None	None
12	ERR3489105	split_err	2020-05-04 01:48:46.013534	2020-05-04 01:48:46.316934	i-077da150eefd54064-1	None	None	None
13	ERR3489106	split_err	2020-05-04 01:48:46.347694	2020-05-04 01:48:46.630278	i-077da150eefd54064-1	None	None	None
14	ERR3489112	split_err	2020-05-04 01:48:46.663209	2020-05-04 01:48:46.981588	i-077da150eefd54064-1	None	None	None
15	ERR3489113	split_err	2020-05-04 01:48:47.017832	2020-05-04 01:48:47.306321	i-077da150eefd54064-1	None	None	None
```

### Block Error
Accession: `SRR11409417`
```
ERROR: Number of bam-blocks in s3://serratus-public/out/200427_CoV_positve_ctrl: 72
is not equal to the expected 79
```